https://docs.smith.langchain.com/tracing/faq/langchain_specific_guides

## Grouping runs from multi-turn interactions

When using LangChain, you may want to group runs from multi-turn interactions together.

With chatbots, copilots, and other common LLM design patterns, users frequently interact with your model over multiple interactions. Each invocation of your model is logged as a separate trace, but you can group these traces together using metadata (see how to add metadata to a run above for more information). Below is a minimal example with LangChain, but the same idea applies when using the LangSmith SDK or API.

In [2]:
# 필요한 모듈과 클래스를 임포트합니다.
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# 대화 프롬프트 템플릿을 생성하고, OpenAI의 GPT-3.5-turbo 모델을 사용하여 대화를 처리하는 체인을 구성합니다.
chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI."),  # 시스템 메시지로 AI의 역할을 정의합니다.
            MessagesPlaceholder(variable_name="chat_history"),  # 대화 기록을 포함하는 자리 표시자입니다.
            ("user", "{message}"),  # 사용자 메시지를 포함합니다. 실제 메시지는 실행 시 결정됩니다.
        ]
    )
    | ChatOpenAI(model="gpt-3.5-turbo")  # OpenAI의 GPT-3.5-turbo 모델을 사용합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
)

# 대화의 고유 식별자를 설정합니다.
conversation_id = "101e8e66-9c68-4858-a1b4-3b0e3c51a933"

# 대화 기록을 저장할 리스트를 초기화합니다.
chat_history = []

# 첫 번째 사용자 메시지를 생성합니다.
message = HumanMessage(content="Hi there")
response = ""

# 체인을 통해 스트리밍 처리를 수행하고, 결과를 출력합니다.
for chunk in chain.stream(
    {
        "message": message,
        "chat_history": chat_history,
    },
    config={"metadata": {"conversation_id": conversation_id}},
):
    print(chunk, end="")
    response += chunk
print()

# 대화 기록에 사용자 메시지와 AI의 응답을 추가합니다.
chat_history.extend(
    [
        message,
        AIMessage(content=response),
    ]
)

# 다음 사용자 메시지가 들어올 경우를 대비한 코드입니다.
next_message = HumanMessage(content="I don't need much assistance, actually.")
for chunk in chain.stream(
    {
        "message": next_message,
        "chat_history": chat_history,
    },
    config={"metadata": {"conversation_id": conversation_id}},
):
    print(chunk, end="")
    response += chunk


Hello! How can I assist you today?
That's perfectly fine! Feel free to reach out if you ever have any questions or need assistance in the future. Have a great day!

In [7]:
# 필요한 모듈과 클래스를 임포트합니다.
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# 대화 프롬프트 템플릿을 생성하고, OpenAI의 GPT-3.5-turbo 모델을 사용하여 대화를 처리하는 체인을 구성합니다.
chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI."),  # 시스템 메시지로 AI의 역할을 정의합니다.
            MessagesPlaceholder(variable_name="chat_history"),  # 대화 기록을 포함하는 자리 표시자입니다.
            ("user", "{message}"),  # 사용자 메시지를 포함합니다. 실제 메시지는 실행 시 결정됩니다.
        ]
    )
    | ChatOpenAI(model="gpt-3.5-turbo")  # OpenAI의 GPT-3.5-turbo 모델을 사용합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
)

# 대화의 고유 식별자를 설정합니다.
conversation_id = "101e8e66-9c68-4858-a1b4-3b0e3c51a933"

# 대화 기록을 저장할 리스트를 초기화합니다.
chat_history = []

while True:
    # 사용자 메시지를 생성합니다.
    message = HumanMessage(content=input("무엇이든 물어보세요\n종료를 원하시면 'd'를 입력하세요"))
    if message.content == 'd':
        break
    print("Human:", message.content)
    print("System: ", end="")
    response = ""

    # 체인을 통해 스트리밍 처리를 수행하고, 결과를 출력합니다.
    for chunk in chain.stream(
        {
            "message": message,
            "chat_history": chat_history,
        },
        config={"metadata": {"conversation_id": conversation_id}},
    ):
        print(chunk, end="")
        response += chunk
    print()

    # 대화 기록에 사용자 메시지와 AI의 응답을 추가합니다.
    chat_history.extend(
        [
            message,
            AIMessage(content=response),
        ]
    )

Human: 답은 2개야
System: 알겠어요. 그 정보를 기억할게요. 다른 도움이 필요하시면 언제든지 말씀해주세요.
Human: 저~기 저~기 산넘고 산넘고 산넘어서 사과나무가 한그루 있다! 거기에 사과가 몇개 열려있게?
System: 사과가 몇 개 열려있느냐는 수수께끼인 것 같네요. 정답은 "2개"입니다. 혹시 다른 수수께끼나 질문이 있으신가요? 함께 풀어보겠습니다!


In [ ]:
# To view all the traces from that conversation in LangSmith, you can query the project using a metadata filter:

# LangSmith
# has(metadata, '{"conversation_id":"101e8e66-9c68-4858-a1b4-3b0e3c51a933"}')

# This will return all traces with the specified conversation ID.

## Getting a run ID from a LangChain call

In Typescript, the run ID is returned in the call response under the __run key. In python, we recommend using the run collector callback. Below is an example:

In [9]:
# langchain 라이브러리에서 chat_models, prompts, callbacks 모듈을 임포트합니다.
from langchain import chat_models, prompts, callbacks

# 대화 체인을 구성합니다.
chain = (
    prompts.ChatPromptTemplate.from_template("Say hi to {name}")  # 사용자 이름을 포함하는 대화 프롬프트 템플릿을 생성합니다.
    | ChatOpenAI(model="gpt-3.5-turbo")
    # | chat_models.ChatAnthropic()  # ChatAnthropic 모델을 사용하여 대화를 처리합니다.
)

# 대화 실행 결과를 수집하기 위해 콜백을 사용합니다.
with callbacks.collect_runs() as cb:
    # 체인을 호출하고, 결과를 저장합니다.
    result = chain.invoke({"name": "Clara"})  # 사용자 이름을 Clara로 설정하여 체인을 호출합니다.
    run_id = cb.traced_runs[0].id  # 실행된 체인의 ID를 가져옵니다.

# 실행된 체인의 ID를 출력합니다.
print(run_id)

72059deb-dd41-4bbb-99cf-a577a3e19897


In [10]:
print(result)

content='Hi Clara! How are you doing today?' response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None} id='run-3a893d41-447e-4f35-a56f-f11ee1369d4b-0'


For python LLMs/chat models, the run information is returned automatically when calling the `generate()` method. Example:

In [11]:
from langchain.chat_models import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
 
chat_model = ChatOpenAI(model="gpt-3.5-turbo") # ChatAnthropic()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a cat"),
        ("human", "Hi"),
    ]
)
res = chat_model.generate(messages=[prompt.format_messages()])
res.run[0].run_id

UUID('59dff906-2a19-4980-8d46-e5cc62c91173')

In [13]:
res

'''
generations=[[
    ChatGeneration(text='Meow! How can I help you today?', 
                   generation_info={'finish_reason': 'stop', 
                                    'logprobs': None}, 
                   message=AIMessage(content='Meow! How can I help you today?', 
                                     response_metadata={'token_usage': {'completion_tokens': 10, 
                                                                        'prompt_tokens': 16, 
                                                                        'total_tokens': 26}, 
                                                        'model_name': 'gpt-3.5-turbo', 
                                                        'system_fingerprint': 'fp_d9767fc5b9', 
                                                        'finish_reason': 'stop', 
                                                        'logprobs': None}, 
                                     id='run-59dff906-2a19-4980-8d46-e5cc62c91173-0'))]] 
llm_output={'token_usage': {'completion_tokens': 10, 
                            'prompt_tokens': 16, 
                            'total_tokens': 26}, 
            'model_name': 'gpt-3.5-turbo', 
            'system_fingerprint': 'fp_d9767fc5b9'}
run=[RunInfo(run_id=UUID('59dff906-2a19-4980-8d46-e5cc62c91173'))]
'''

generations=[[ChatGeneration(text='Meow! How can I help you today?', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Meow! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 16, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}, id='run-59dff906-2a19-4980-8d46-e5cc62c91173-0'))]] llm_output={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 16, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9'} run=[RunInfo(run_id=UUID('59dff906-2a19-4980-8d46-e5cc62c91173'))]


In [15]:
res.generations[0][0].message.id

'run-59dff906-2a19-4980-8d46-e5cc62c91173-0'

or for LLMs

In [16]:
from langchain.llms import OpenAI

openai = OpenAI()
res = openai.generate(["You are a good bot"])
print(res.run[0].run_id)

c:\Users\r2com\Desktop\kpopmap\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


7e4c0e15-858e-47b8-9420-d2c67daecd26


In [17]:
res

LLMResult(generations=[[Generation(text='.\n\nThank you! I am programmed to assist and provide helpful responses. Is there anything specific you need assistance with?', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 29, 'prompt_tokens': 5, 'completion_tokens': 24}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('7e4c0e15-858e-47b8-9420-d2c67daecd26'))])

# 잘 모르겠음
LangSmith 아직 공부 안했고,, 기능도 모르겠음

## Tracing without environment variables

In [19]:
from langchain.callbacks import LangChainTracer
from langchain_openai import ChatOpenAI
# from langsmith import Client

callbacks = [
  LangChainTracer(
    project_name="YOUR_PROJECT_NAME_HERE",
    # client=Client(
    #   api_url="https://api.smith.langchain.com",
    #   api_key="YOUR_API_KEY_HERE"
    # )
  )
]

llm = ChatOpenAI()
llm.invoke("Hello, world!", config={"callbacks": callbacks})

LangSmithUserError: API key must be provided when using hosted LangSmith API

This tactic is also useful for when you have multiple chains running in a shared environment but want to log their run traces to different projects.

## Ensuring all traces are submitted before exiting

In LangChain Python, LangSmith's tracing is done in a background thread to avoid obstructing your production application. This means that your process may end before all traces are successfully posted to LangSmith. This is especially prevalent in a serverless environment, where your VM may be terminated immediately once your chain or agent completes.

In LangChain JS, the default is to block for a short period of time for the trace to finish due to the greater popularity of serverless environments. You can make callbacks asynchronous by setting the LANGCHAIN_CALLBACKS_BACKGROUND environment variable to "true".

For both languages, LangChain exposes methods to wait for traces to be submitted before exiting your application. Below is an example:

In [18]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

llm = ChatOpenAI()
try:
    llm.invoke("Hello, World!")
finally:
    wait_for_all_tracers()